# Uploading Data Files

## Extract Insects species and genus names

In [1]:
import pandas as pd
insects=pd.read_excel('Insect codes with names.xls')
genus_all=list(insects['GENUS'])
insects_all=list(insects['SPECIES'])
insects=[]
for key,value in enumerate(genus_all):
    insects.append(f"{str(value).strip()} {str(insects_all[key]).strip()}")
len(insects_all)
    
    # combined details
    # https://boldsystems.org/index.php/API_Public/cobined?taxon=Abutilon%20hirtum

595

In [3]:
insects[0:5]

['Agathis sp.', 'nan nan', 'Alcidodes sp.', 'Aleiodes sp.', 'nan nan']

## Extract plant species and genus names 

In [5]:
import pandas as pd
df= pd.read_excel('master_file_updated_may_version1.xlsx')
plant_species_list=[]
species_all=list(df['Plant species with new names merged'])
for i in species_all:
    name=f"{i.split()[0].strip()} {i.split()[1].strip()}"
    plant_species_list.append(name)    # print(i)

# species_list

In [6]:
len(set(species_all))

873

# Final barcode mining 

In [7]:
# code with summary statistics
import io
import pandas as pd
import requests
no_marker=[]
no_matk=[]
def barcode_from_bold(species_list, barcode_marker):
    # API endpoint
    api_url = 'http://www.boldsystems.org/index.php/API_Public/combined?'

    # Parameters
    params = {
        'format': 'tsv',
        'data_type': 'DNA',
        # 'marker': barcode_marker,
    }

    # Create an empty DataFrame to hold the results
    results_df = pd.DataFrame()

    # Create counters for summary statistics
    total_species = len(species_list)
    count_no_data = 0
    count_no_marker = 0

    # Loop over each species and fetch its data
    for species in species_list:
        print(species)
        # Set the 'taxon' parameter to the current species name
        params['taxon'] = species
        try:
            response = requests.get(api_url, params=params, timeout=50)  # Timeout set to 10 seconds
            if response.status_code == 200:
                data = response.text
                if len(data) != 0:
                    if barcode_marker in data:  # Check if the marker is present in the response data
                        try:
                            # Parse the TSV data into a pandas DataFrame
                            df = pd.read_csv(io.StringIO(data), sep='\t', usecols=['genbank_accession', 'species_name', 'country', 'marker_codes', 'nucleotides', 'processid'])
                            # df = pd.read_csv(io.StringIO(data), sep='\t', usecols=['genbank_accession', 'species_name', 'country', 'marker_codes', 'nucleotides'])

                            # Append the data for this species to the results DataFrame
                            results_df = pd.concat([results_df, df], ignore_index=True)
                        except:
                            no_marker.append(species)

                            print(f"No data received for {species}")
                            count_no_data += 1
                            continue
                    else:
                        no_matk.append(species)

                        print(f"Marker not found for {species}.")
                        count_no_marker += 1
                else:
                    no_marker.append(species)

                    print(f"No data received for {species}")
                    count_no_data += 1
            else:
                print(f"Error fetching data for {species}. Status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {species}. Error: {e}")
            continue

    # Add the species ID column
    results_df['species_id'] = results_df['species_name'].apply(lambda x: species_list.index(x) + 1 if x in species_list else None)

    # Filter the rows based on the marker code
    results_df = results_df[results_df['marker_codes'].str.contains(barcode_marker) | results_df['marker_codes'].isna()]
    print(results_df)

    # Drop the marker_codes column
    results_df.drop(columns=["marker_codes"], inplace=True)

    # Save the DataFrame to a CSV file
    results_df.to_csv(f"{barcode_marker}.csv", index=False)

    # Save summary statistics to a separate file
    summary_stats = {
        'Total Species': [total_species],
        'Count No Data': [count_no_data],
        'Count No matK marker but other markers': [count_no_marker]
    }
    summary_df = pd.DataFrame(summary_stats)
    summary_df.to_csv(f"{barcode_marker}_summary.csv", index=False)


In [8]:
# plant_species_list

In [12]:
# Plants matk
species_list = plant_species_list
df=barcode_from_bold(species_list,"matK")
df

Abutilon hirtum
Abutilon mauritianum
Acalypha neptunica
No data received for Acalypha neptunica
Acanthopale pubescens
No data received for Acanthopale pubescens
Acanthus eminens
No data received for Acanthus eminens
Acanthus polystachius
No data received for Acanthus polystachius
Achyrospermum schimperi
No data received for Achyrospermum schimperi
Acokanthera oppositifolia
Acokanthera schimperi
No data received for Acokanthera schimperi
Adansonia digitata
Adenia bequaertii
No data received for Adenia bequaertii
Adenia gummifera
Adenia lobata
Marker not found for Adenia lobata.
Adenia wightiana
Marker not found for Adenia wightiana.
Adenium obesum
Aframomum angustifolium
Aframomum zambesiacum
No data received for Aframomum zambesiacum
Afzelia quanzensis
Agelaea pentagyna
No data received for Agelaea pentagyna
Agelanthus sansibarensis
Agelanthus sp.
No data received for Agelanthus sp.
Agelanthus zizyphifolius
No data received for Agelanthus zizyphifolius
Alafia caudata
Marker not found f

## Load the matk and create a table matching that in the matk table database

In [33]:
df_matk=pd.read_csv('matk_final_sep.csv')
# Read the CSV file into a DataFrame
df_matk = pd.read_csv('matk_final_sep.csv')

# Add the 'matk_id' column starting from 1
df_matk['matk_id'] = range(1, len(df_matk) + 1)
# Rename the 'species' column to 'plant_id'
df_matk = df_matk.rename(columns={'species_id': 'plant_id'})
df_matk = df_matk.rename(columns={'nucleotides': 'nucleotide'})
df_matk = df_matk.rename(columns={'processid': 'bold_id'})

df_matk
df_matk=df_matk[["matk_id","plant_id","nucleotide","country","genbank_accession","bold_id"]]
# Save the DataFrame with the new column back to a CSV file
df_matk.to_csv('matk_final_sep_with_database_columns.csv', index=False)
df_matk

,matk_id,plant_id,nucleotide,country,genbank_accession,bold_id
0,1,1,GCTTCTTCTTTGCATTTATTACGGTTCTCTCTCTACGAGTATTGTA...,Kenya,KR734379,UHURU142-14
1,2,1,TTCTTCTTTGCATTTATTACGGTTCTCTCTCTACGAGTATTGTAAT...,Kenya,KR734637,UHURU143-14
2,3,1,GCTTCTTCTTTGCATTTATTACGGTTCTCTCTCTACGAGTATTGTA...,Kenya,KR734738,UHURU144-14
3,4,2,CCTATATAATTCTCATGTATGTGAATACGAGTCCATTTTCCTTTTT...,Kenya,KR734566,UHURU948-14
4,5,2,CTTCTCATTTACGATCAACATCTTCTGGAGTCTTTCTTGAACGAAT...,Kenya,KR734848,UHURU949-14
...,...,...,...,...,...,...
680,681,880,ACTCTTTTTTTTTGAGGATCCGCTGAAATAATGAGAAAGATGTCTG...,Mozambique,NaN,PNG094-18
681,682,880,CTTTGCATTTATTACGGCTTTTTCTTCACGAGTATTATAATTGGAA...,Kenya,MK285045,UHURU1619-15
682,683,881,---------------------------TCTTCTTTGCATTTATTAC...,Mozambique,JX517471,TSA107-10
683,684,882,AATTATGTATCAGATGTACGAATACCCTACCCCATTCATCTGGAAA...,United States,OL690208,GRDMO281-21


In [13]:
# species_list = plant_species_list
# d=barcode_from_bold(['Warburgia ugandensis'],'matK')
# plant_species_list[0]

In [11]:
len(insects_updated)

183

# Downloading Insects Barcode

In [129]:
def barcode_from_bold_insects(insect_species, barcode_marker):
    "input: list of species name\
           :barcode marker-user cn specify marker as either maturaseK,CoI or Rbcl\
    output: a csv file with colums of species id as obtained from the original master file,the barcode sequence, accession and country of collection" 
    import io
    import pandas as pd
    import requests

    # API endpoint
    api_url = 'http://www.boldsystems.org/index.php/API_Public/combined?'

    # Parameters
    params = {
        'format': 'tsv',
        'data_type': 'DNA',
    }

    # Create an empty DataFrame to hold the results
    results_df = pd.DataFrame()
    # Create counters for summary statistics
    total_species = len(insect_species)
    count_no_data = 0
    count_no_marker = 0

    # Loop over each species and fetch its data
    for species in insect_species:
        # if str(species).startswith("Fopius")
        print(species)
        # Set the 'taxon' parameter to the current species name
        params['taxon'] = species
        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = response.text
            if len(data) != 0:
                try:
                    # Parse the TSV data into a pandas DataFrame
                    df = pd.read_csv(io.StringIO(data), sep='\t', usecols=['genbank_accession', 'species_name', 'country', 'marker_codes', 'nucleotides', 'processid'])
                    # df = pd.read_csv(io.StringIO(data), sep='\t', usecols=['genbank_accession', 'species_name', 'country', 'marker_codes', 'nucleotides'])
                    df = pd.read_csv(io.StringIO(data), sep='\t')

                    # print(df['marker_codes'].str.split())
  # Print the values in the 'marker_codes' column
                    # print(df)  # Print the filtered dataframe

                    # Append the data for this species to the results DataFrame
                    results_df = pd.concat([results_df, df], ignore_index=True)
                    results_df = results_df[results_df['marker_codes'].str.contains(barcode_marker)]

                except:
                    pass
            else:
                print(f"No data received for {species}")
                continue
        else:
            print(f"Error fetching data for {species}. Status code: {response.status_code}")
            pass



   # Add the species ID column
    results_df['species_id'] = results_df['species_name'].apply(lambda x: insects.index(x) + 1 if x in insects else None)

    # Function to update species name if it matches 'Blastobasis kenya'
    def update_species_name(name):
        if name == 'Blastobasis kenya':
            return 'Blastobasis kenya B3A'
        else:
            return name

    # Apply the update_species_name function to update the species names
    results_df['species_name'] = results_df['species_name'].apply(update_species_name)

    # Add the 'species_id' column based on the updated species names
    results_df['species_id'] = results_df['species_name'].apply(lambda x: insects.index(x) + 1 if x in insects else None)

    # Filter the rows based on the marker code
    results_df = results_df[results_df['marker_codes'].str.contains(barcode_marker) | results_df['marker_codes'].isna()]
    # print(results_df)

    # Drop the marker_codes column
    results_df.drop(columns=["marker_codes"], inplace=True)

    # Save the DataFrame to a CSV file
    results_df.to_csv(f"{barcode_marker}.csv", index=False)

    # Save summary statistics to a separate file
    summary_stats = {
        'Total Species': [total_species],
        'Count No Data': [count_no_data],
        'Count No matK marker but other markers': [count_no_marker]
    }
    summary_df = pd.DataFrame(summary_stats)
    summary_df.to_csv(f"{barcode_marker}_summary.csv", index=False)
        # Print the reults dataframe
    return results_df

# Example usage
df=barcode_from_bold_insects(insects_updated, 'COI-5P')
df

Araecerus fascicularis
No data received for Araecerus fascicularis
Blastobasis kenya
Bactrocera amplexa
Bactrocera bigutulla
No data received for Bactrocera bigutulla
Bactrocera munroi
Bactrocera munroi
Bactrocera cucurbitae
No data received for Bactrocera cucurbitae
Bactrocera nigrivenata
Bactrocera oleae
Bactrocera invadens
Brachypeplus depressus
No data received for Brachypeplus depressus
Capparimyia aenigma
Capparimyia bipustulata
Capparimyia melanaspis
Carpophthoromyia dimidiata
Carpophilus fumatus
No data received for Carpophilus fumatus
Carpomya incompleta
Carpophilus dimidiatus
Hendecasis cf apicefulva
No data received for Hendecasis cf apicefulva
Hendecasis cf duplifascialis
No data received for Hendecasis cf duplifascialis
Prochoristis calamochroa
Ceratitis anonae
Ceratitis argentobrunnea
No data received for Ceratitis argentobrunnea
Ceratitis perisae
No data received for Ceratitis perisae
Ceratitis bremii
Ceratitis perseus
Ceratitis caetrata
Ceratitis capitata
Ceratitis conn

,processid,sampleid,recordID,catalognum,fieldnum,institution_storing,collection_code,bin_uri,phylum_taxID,phylum_name,...,genbank_accession,nucleotides,trace_ids,trace_names,trace_links,run_dates,sequencing_centers,directions,seq_primers,species_id
0,AFMIC184-12,USNM ENT 00808072,2506399,USNM ENT 00808072,NaN,"Smithsonian Institution, National Museum of Na...",NaN,BOLD:AAA6731,20,Arthropoda,...,KF643041,AACCTTATATTTTATTTTTGGAATTTGAGCTGGAATAGTAGGAACC...,4037603|4037413,"AFMIC184-12[LepF1,LepR1]_F.ab1|AFMIC184-12[Lep...",http://trace.boldsystems.org/traceIO/bold.org/...,2012-05-02 13:31:15|2012-05-02 14:57:48,Centre for Biodiversity Genomics|Centre for Bi...,F|R,LepF1|LepR1,21.0
1,AFRM012-12,USNM ENT 00808090,2508078,USNM ENT 00808090,NaN,"Smithsonian Institution, National Museum of Na...",NaN,NaN,20,Arthropoda,...,KF643039,AACCTTATATTTTATTTTTGGAATTTGAGCTGGAATAGTAGGAACC...,4157647|4119324|4119287,"AFRM012-12[MLepF1,LepR1]_R.ab1|AFRM012-12[LepF...",http://trace.boldsystems.org/traceIO/bold.org/...,2012-05-29 23:10:47|2012-05-19 02:27:56|2012-0...,Centre for Biodiversity Genomics|Centre for Bi...,R|R|F,LepR1|MLepR2|LepF1,21.0
2,AFRM023-12,USNM ENT 00808101,2508089,USNM ENT 00808101,NaN,"Smithsonian Institution, National Museum of Na...",NaN,BOLD:AAA6731,20,Arthropoda,...,KF643042,----------------------------------------------...,4157655|4157618|4119332|4119295|4037261|4037210,"AFRM023-12[MLepF1,LepR1]_R.ab1|AFRM023-12[MLep...",http://trace.boldsystems.org/traceIO/bold.org/...,2012-05-29 23:10:47|2012-05-30 02:35:48|2012-0...,Centre for Biodiversity Genomics|Centre for Bi...,R|F|R|F|R|F,LepR1|MLepF1|MLepR2|LepF1|LepR1|LepF1,21.0
3,AFRM028-12,USNM ENT 00808106,2508094,USNM ENT 00808106,NaN,"Smithsonian Institution, National Museum of Na...",NaN,BOLD:AAA6731,20,Arthropoda,...,KF643038,AACCTTATATTTTATTTTTGGAATTTGAGCTGGAATAGTAGGAACC...,4157657|4157620|4119334|4119297,"AFRM028-12[MLepF1,LepR1]_R.ab1|AFRM028-12[MLep...",http://trace.boldsystems.org/traceIO/bold.org/...,2012-05-29 23:10:47|2012-05-30 02:35:48|2012-0...,Centre for Biodiversity Genomics|Centre for Bi...,R|F|R|F,LepR1|MLepF1|MLepR2|LepF1,21.0
4,HCBK009-03,USNM ENT 00196870,4495,USNM ENT 00196870,NaN,Smithsonian Institution,NaN,BOLD:AAA6731,20,Arthropoda,...,GQ330222,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNCTGGATAGTAGGAACC...,195283|195282,HCBK009-03MR.ab1|HCBK009-03MF.ab1,http://trace.boldsystems.org/traceIO/bold.org/...,2004-12-23 07:58:34|2004-12-23 07:58:34,Centre for Biodiversity Genomics|Centre for Bi...,F|F,MLepF1|LepF1,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3197,PMANK904-08,USNM ENT 00678366,917055,USNM ENT 00678366,USNM ENT 00678366,"Smithsonian Institution, National Museum of Na...",NaN,BOLD:AAE0418,20,Arthropoda,...,KF643201,AACTTTATATTTTATTTTTGGAATTTGATCAGGAATAGTGGGAACA...,1212294|1212199,PMANK904-08_R.ab1|PMANK904-08_F.ab1,http://trace.boldsystems.org/traceIO/bold.org/...,2009-01-06 19:05:42|2009-01-06 17:34:27,Centre for Biodiversity Genomics|Centre for Bi...,R|F,LepR1|LepF1,593.0
3198,PMANK913-08,USNM ENT 00678378,917064,USNM ENT 00678378,USNM ENT 00678378,"Smithsonian Institution, National Museum of Na...",NaN,BOLD:AAE0418,20,Arthropoda,...,KF643206,AACTTTATATTTTATTTTTGGAATTTGATCAGGAATAGTGGGAACA...,1212303|1212208,PMANK913-08_R.ab1|PMANK913-08_F.ab1,http://trace.boldsystems.org/traceIO/bold.org/...,2009-01-06 19:05:42|2009-01-06 17:34:27,Centre for Biodiversity Genomics|Centre for Bi...,R|F,LepR1|LepF1,593.0
3199,PMANK916-08,USNM ENT 00678384,917067,USNM ENT 00678384,USNM ENT 00678384,"Smithsonian Institution, National Museum of Na...",NaN,BOLD:AAE0418,20,Arthropoda,...,KF643208,AACTTTATATTTTATTTTTGGAATTTGATCAGGAATAGTGGGAACA...,1212306|1212211,PMANK916-08_R.ab1|PMANK916-08_F.ab1,http://trace.boldsystems.org/traceIO/bold.org/...,2009-01-06 19:05:42|2009-01-06 17:34:27,Centre for Biodiversity Genomics|Centre for Bi...,R|F,LepR1|LepF1,593.0
3200,PMANL2551-12,USNM ENT 00808488,2715166,USNM ENT 00808488,NaN,"Smithsonian I